In [1]:
import sys
import argparse
import os
import pandas as pd
import numpy as np
import re
from collections import Counter

sys.path.append('/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts')
import configuration_util
import analysis_utils 
import mutationSigUtils 
import maf_analysis_utils
import mutation_modeling_util
import get_gene_and_cohort_list_utils
import clonality_analysis_util
import math

filePathDict = configuration_util.get_all_files_path_dict()

In [3]:
phasingData = pd.read_table(filePathDict['PHASING_DATA'])

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [4]:
allHypermutantIds = get_gene_and_cohort_list_utils.get_all_hypermutant_ids(
    hypermutantIdDir=filePathDict['HYPERMUTATION_STATUS_IDS'])
phasingHyper = phasingData[phasingData['Tumor_Sample_Barcode'].isin(allHypermutantIds)]

/Users/friedman/Desktop/hypermutationProjectFinal/scripts/utilityScripts/get_gene_and_cohort_list_utils.py:170: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  df = pd.read_table(path)


In [5]:
#reload(clonality_analysis_util) #for some reason this isnt working
whitelistFlatGenomeCases = clonality_analysis_util.get_facets_whitelist()
phasingHyper['adjPhase'] = phasingHyper.apply(lambda row: 
    'trans' if row['phase'] == 'trans or separate cells' and row['Tumor_Sample_Barcode'] in whitelistFlatGenomeCases
    else row['phase'], axis=1)

/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [32]:
#dictionaryOfRelatedGenes = get_gene_and_cohort_list_utils.get_related_genes_by_cancer_type()

In [34]:
#Prepare data for plotting
plotThresh = 5
genesToHighlight = [key for key, value in dict(phasingHyper[(phasingHyper['oncogenic.1'].notnull()) & (phasingHyper['oncogenic.2'].notnull()) & (phasingHyper['adjPhase'].isin(['cis', 'trans']))][
    'Hugo_Symbol'].value_counts()).items() if value > plotThresh]

phasingHyper['label'] = phasingHyper.apply(lambda row: 
                                           '1 or 2 silent' if (row['Variant_Classification.1'] == 'Silent') | (row['Variant_Classification.2'] == 'Silent')
                                           else '1 or 2 VUS'
                                               if (not isinstance(row['oncogenic.1'], str)) | (not isinstance(row['oncogenic.2'], str))
                                           else row['Hugo_Symbol'] if row['Hugo_Symbol'] in genesToHighlight
                                           
                                           else 'related_tsg' if row['Role'] == 'TSG' and row['CANCER_TYPE'] in dictionaryOfRelatedGenes and row['Hugo_Symbol'] in dictionaryOfRelatedGenes[row['CANCER_TYPE']]
                                           else 'related_oncogene' if row['Role'] != 'TSG' and row['CANCER_TYPE'] in dictionaryOfRelatedGenes and row['Hugo_Symbol'] in dictionaryOfRelatedGenes[row['CANCER_TYPE']]
                                           
                                           else 'other_tsg' if row['Role'] == 'TSG'
                                           else 'other_oncogene' if row['Role'] == 'Oncogene'
                                           else 'other', axis=1)

phasingHyper['isTrans'] = phasingHyper['adjPhase'].apply(lambda x: 1 if x == 'trans' else 0 if x == 'cis' else None)
phasingHyperWrite = phasingHyper[phasingHyper['isTrans'].notnull()]


/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/anaconda2/lib/python2.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [35]:
writeDir = os.path.join(os.getcwd(), 'FIGURE3_PLOTTING_FILES')
phasingHyperWrite.to_csv('/Users/friedman/Desktop/hypermutationProjectFinal/files/infoFiles/phasingSummary.tsv',
                         index=False, sep='\t')

In [15]:
#Counter(phasingHyper['label'])

Counter(phasingHyper[(phasingHyper['isTrans'].notnull()) & (phasingHyper['oncogenic.1'].notnull()) & (phasingHyper['oncogenic.2'].notnull())
                    ]['Hugo_Symbol']).most_common(10)

#phasingHyper['oncogenic.1']

[('TP53', 42),
 ('TERT', 39),
 ('PTEN', 29),
 ('PIK3CA', 18),
 ('B2M', 14),
 ('APC', 9),
 ('ARID1A', 8),
 ('INPPL1', 5),
 ('PIK3R1', 5),
 ('CDKN2A', 4)]